In [70]:
# Katherine Yohanna Mazariegos Guerra
# Abner Xocop Chacach

# En archivo exercise01.txt x1, x2 y x3 es el nombre de las variables, se usaron esas para seguir con el estándar de nombramiento en Programación Lineal.
# Es necesario escribir si es min/max para que el programa identifique si es de maximización o minimización, st para indicar que esas son las restricciones y end para indicar al programa que el problema termina ahí

def parse_coefficients(coefficient_list, monomial):
    """
    Este es un parseador de coeficientes. Consiste en comprobar si una cadena tiene una expresión regular en donde pueda extraer caracteres específicos, en este caso se busca extraer los coeficientes.

    Args:
        :rtype: None
        :param coefficient_list: Lista en la que se almacenarán los coeficientes
        :param monomial: Una cadena (por ejemplo, -3x1) que será analizada hasta su coeficiente (por ejemplo, -3)

    Verifica qué patrón coincide. Válidos son: (s)(n)lv
        Los paréntesis indican la existencia opcional
        s es + o - (la ausencia significa +)
        n es un número (coeficiente, la ausencia significa 1)
        l es una letra latina minúscula (letra variable)
        v es un número, probablemente incremental (número variable)

    Import re:
        Una expresión regular (o RE) especifica un conjunto de cadenas que se corresponde con ella; las funciones de este módulo le permiten comprobar si una cadena particular se corresponde con una expresión regular dada (o si una expresión regular dada se corresponde con una cadena particular, que se reduce a lo mismo)
        Source: https://docs.python.org/3/library/re.html
    """
    import re

    if re.match('[ ]*[\+ ]?[\d]+[\.]?[\d]*', monomial):
        float_cast = float(re.match('[ ]*[\+ ]?[\d]+[\.]?[\d]*', monomial).group(0))
        coefficient_list.append(float_cast)
    elif re.match('[ ]*[\-][\d]+[\.]?[\d]*', monomial):
        float_cast = float(re.match('[ ]*[\-][\d]+[\.]?[\d]*', monomial).group(0))
        coefficient_list.append(float_cast)
    elif re.match('[ ]*[\+]*[a-z][\d]+', monomial):
        coefficient_list.append(1)
    elif re.match('[ ]*[\-][a-z][\d]+', monomial):
        coefficient_list.append(-1)

In [71]:
def parse_lp1(input_filename):
    """
    Esta función es la encargada de leer el archivo, hará uso del parser para mandar línea a línea el contenido del .txt y según los coeficientes que obtenga devolverá las matrices y arrays correspondientes. 

    Tiene tareas como verificar que el archivo haya sido encontrado, leer si es un problema de maximización o minimización, llenar las matrices/ arrays.

    :rtype : tuple
    :param input_filename: Nombre de archivo de la entrada del problema lineal
    :return: Retorna A-matrix, b-vector, c-vector, MinMax
    """
    import re

    """
    Inicializar la variable de error
    Si el error!=0 entonces hubo un problema de entrada de archivo  
    """
    error = 0

    try:
        infile = open('exercise01.txt')
    except FileNotFoundError:
        error = 1
        print('\nInput file error: Archivo no encontrado.') # Archivo no encontrado

    lines = []
    if error != 1:
        for line in infile:
            lines.append(line)
        infile.close()
    for line in lines:
        print(line, end='')

    """
    Verficar si el problema es de maximización o de minimización
    """ 
    minmax_line = ''
    for line in lines:
        if re.match('^[ ]*max|min', line):
            minmax_line = line

    minmax = 0
    objective_function = ''

    """
    Si en el archivo se encuentra la palabra 'max' entonces el problema es de maximización
    """
    if re.match('^[ ]*max', minmax_line):
        minmax = 1
        objective_function = minmax_line
        objective_function = objective_function.strip('max')

    # Si en el archivo se encuentra la palabra 'min' entonces el problema es de minimización
    
    elif re.match('^[ ]*min', minmax_line):
        minmax = -1
        objective_function = minmax_line
        objective_function = objective_function.strip('min')

    """
    Si en el archivo no se encuentra ni 'max' ni  'min' entonces no hay función objetivo
    """
    if minmax_line == '' and minmax == 0:
        error = 2
        print('\nInput file error: Función objetivo no encontrada.')

    """
    Rellenar el vector c con coeficientes de función objetiva
    """
    c_vector = []

    regex = re.compile('^[\+\- ]?[\d]*[\.]?[\d]*[a-z][\d+]')
    while regex.match(objective_function):
        monomial = regex.match(objective_function).group(0)
        parse_coefficients(c_vector, monomial)
        objective_function = objective_function.replace(monomial, '', 1)

    """
    Rellenar la matriz A (coeficientes) y el vector b utilizando las restricciones del problema
    """
    a_matrix = []
    b_vector = []
    eqin = []

    st_line = ''
    st_index = 0
    for index, line in enumerate(lines):
        if 'st' in line:
            st_index = index
            st_line = line

    if re.match('^[ ]*st', st_line):
        st_line = st_line.replace('st', '  ', 1)

    if st_line == '':
        error = 3
        print('\nInput file error: Línea de restricciones no encontrada. No existe la keyword \'st\'.')

    while st_index < len(lines) - 1:
        sub_a_vector = []
        a_matrix.append(sub_a_vector)
        while True:
            st_line = st_line.strip(' ')
            if re.match('^[\+\- ]?[\d]*[\.]?[\d]*[a-z][\d+]', st_line):
                monomial = re.match('^[\+\- ]?[\d]*[\.]?[\d]*[a-z][\d+]', st_line).group(0)
                parse_coefficients(sub_a_vector, monomial)
                st_line = st_line.replace(monomial, '', 1)
            elif re.match('^[<>=]+', st_line):
                monomial = re.match('^[<>=]+', st_line).group(0)
                if monomial == '<=':
                    eqin.append(-1)
                elif monomial == '>=':
                    eqin.append(1)
                elif monomial == '=':
                    eqin.append(0)
                else:
                    error = 4
                    print('\nInput file error: Caracter inesperado; esperados <=, >=, = al menos', monomial)
                st_line = st_line.replace(monomial, '', 1)
            elif re.match('^[\d]+', st_line):
                monomial = re.match('^[\d]+', st_line).group(0)
                int_cast = int(re.match('^[\d]+', st_line).group(0))
                b_vector.append(int_cast)
                st_line = st_line.replace(monomial, '', 1)
            else:
                if not sub_a_vector:    # Evalúa true cuando las líneas están vacías entre las restricciones
                    a_matrix.pop()
                break

        """
        Incrementar el número de línea y obtener el siguiente
        """
        st_index += 1
        st_line = lines[st_index]

        """
        Búsqueda de la declaración final y ausencia de errores
        """
        if st_line == 'end\n' and error == 0:
            print('\nArchivo cargado exitosamente.')
            break


    """
    Devolver todas las listas y variables creadas
    """
    return a_matrix, b_vector, c_vector, eqin, minmax

In [72]:
def convert_to_dual(input_filename, output_filename):
    """
    Verifica si son restricciones de >=, <= o =. También tiene como tarea hacer un archivo de salida en el que muestre los resultados de las matrices que se llenaron.

    :param input_filename: Nombre de archivo de la entrada del problema lineal
    :param output_filename: Filename of the linear problem output
    :return: Returns A-matrix, b-vector, c-vector, Variable-constraints, MinMax
    """

    # Llamar la función parse_lp1
    (a_matrix, b_vector, c_vector, eqin, minmax) = parse_lp1(input_filename)

    """
    Convertir las restricciones a equivalentes duales
    '*' significa libre
    """
    variable_constraints = []
    if minmax == -1:
        for el in eqin:
            if el == 0:
                variable_constraints.append('*')
            elif el == 1:
                variable_constraints.append('>=0')
            elif el == -1:
                variable_constraints.append('<=0')

    """
    Traspuesta de A-matrix
    """
    a_matrix = list(zip(a_matrix))

    """
    min(max) el problema dual es max(min)
    """
    minmax = -minmax

    """
    Escribir el problema a un archivo de salida
    """
    outfile = open(output_filename, 'w')
    outfile.write('(Objective Function) b-vector: [' + ', '.join(map(str, b_vector)) + ']\n')

    outfile.write('\nA-matrix: [')
    thing = ''
    for index, sub_a_vector in enumerate(a_matrix):
        thing += '[ ' + ', '.join(map(str, sub_a_vector)) + ']'
        if index != (len(a_matrix) - 1):
            thing += ', '
    outfile.write(thing + ']\n')

    outfile.write('\n(Contraints) c-vector: [' + ', '.join(map(str, c_vector)) + ']\n')
    outfile.write('\n(Variable Contraints) variable_constraints-vector: [' + ', '.join(map(str, c_vector)) + ']\n')
    outfile.write('\nEqin: [' + ', '.join(map(str, eqin)) + ']\n')
    outfile.write('\nMinMax: [' + str(minmax) + ']\n')
    outfile.close()

    return a_matrix, b_vector, c_vector, variable_constraints, eqin, minmax


(a_matrix, b_vector, c_vector, variable_contraints, eqin, minmax) = convert_to_dual('input-lp1', 'output-lp2')

"""
    Se hará una comprobación para que cuando se imprima muestre si es maximización o minimización
"""

print('\n\n=== Resultados ===')

if minmax == 1:
    minmaxStr = 'Maximización'
else: minmaxStr = 'Minimización'

print('c-vector:', c_vector)
print('A-matrix:', a_matrix)
print('b-vector:', b_vector)
print('Variable-contraints-vector:', variable_contraints)
print('Eqin:', eqin)
print('MinMax:', minmaxStr)

min x1+x2-4x3
st x1+x2+2x3<=9
   x1-3x2-x3<=20
   -x1+x2+5x3<=4
end

=== Resultados ===
c-vector: [1, 1, -4.0]
A-matrix: [([1, 1, 2.0],), ([1, -3.0, -1],), ([-1, 1, 5.0],)]
b-vector: [9, 20, 4]
Variable-contraints-vector: ['<=0', '<=0', '<=0']
Eqin: [-1, -1, -1]
MinMax: Maximización


In [38]:
import numpy as np

In [43]:
x = (np.linalg.inv(a_matrix))*b_vector
print(x)

LinAlgError: Last 2 dimensions of the array must be square